In [1]:
import boolexpr as bx
import itertools

In [ ]:
n = 113



# This version looks for full reps of 33_65 over Z/nZ

G = list(range(n))
G_star = list(range(1,n))
ctx = bx.Context()
variables = { **{ (x,0) : ctx.get_var(str((x,0))) for x in G_star} ,
             **{ (x,1) : ctx.get_var(str((x,1))) for x in G_star} ,  
             **{ (x,2) : ctx.get_var(str((x,2))) for x in G_star} }


# exactly one color is true
conjuncts_colors = ( (variables[(x,0)] | variables[(x,1)] | variables[(x,2)]) 
                    & (~variables[(x,0)] | ~variables[(x,1)]) 
                    & (~variables[(x,0)] | ~variables[(x,2)]) 
                    & (~variables[(x,1)] | ~variables[(x,2)])  
                    for x in G_star )

phi_0 = bx.and_(*conjuncts_colors)

# red must be symmetric
red_inverse_pairs = ( bx.eq(variables[(x,0)],variables[((-x)%n,0)] )
                 for x in G_star )

# b1 must be symmetric
blue_inverse_pairs = ( bx.eq(variables[(x,1)],variables[((-x)%n,1)] )
                 for x in G_star )

# b2 must be symmetric
green_inverse_pairs = ( bx.eq(variables[(x,2)],variables[((-x)%n,2)] )
                 for x in G_star )



phi_1 = bx.and_( bx.and_(*red_inverse_pairs), bx.and_(*blue_inverse_pairs), bx.and_(*green_inverse_pairs) )


def disjunction(x,c1,c2):
    # returns a disjunction that says x in G_star has need c1-c2 met
    witnesses = {(y,(x-y)%n) for y in set(G_star) - {x}}
    return bx.or_( *( variables[(y, c1)] 
                               & variables[(z, c2)] 
                     for y,z in witnesses ) ) 

red_needs = {(0,0), (0,1), (0,2), (1,0), (2,0), (1,1), (1,2), (2,1), (2,2)}

blue_needs = {(0,0), (0,1), (1,0), (0,2), (2,0), (1,1)}

green_needs = {(0,0), (0,1), (0,2), (1,0), (2,0)}

conjuncts_red_needs = ( bx.impl(variables[(x,0)] , 
                            bx.and_(*( disjunction(x,c1,c2) for c1,c2 
                              in red_needs ) ) )  
                             for x in G_star )

conjuncts_blue_needs = ( bx.impl(variables[(x,1)] , 
                            bx.and_(*( disjunction(x,c1,c2) for c1,c2 
                              in blue_needs ) ) )  
                             for x in G_star )


conjuncts_green_needs = ( bx.impl(variables[(x,2)] , 
                            bx.and_(*( disjunction(x,c1,c2) for c1,c2 
                              in green_needs ) ) )  
                             for x in G_star )




phi_2 = bx.and_( bx.and_(*conjuncts_red_needs), 
                bx.and_(*conjuncts_blue_needs),  
                bx.and_(*conjuncts_green_needs) )

conjuncts_no_forb_triangles = ( bx.impl(~(variables[(i,0)] | variables[(j,0)] 
                               | variables[( (i+j)%n,0)] ), 
                                       (variables[(i,1)] & variables[(j,1)] 
                                       & variables[( (i+j)%n,1)] ) )
                    for i,j in itertools.combinations_with_replacement(G_star,2) 
                               if (i+j)%n != 0 )

phi_3 = bx.and_(*conjuncts_no_forb_triangles)


# phi = bx.and_(phi_0, phi_1, phi_2)
phi = bx.and_(phi_0, phi_1, phi_2, phi_3)

phi.tseytin(ctx)

phi.sat()

In [11]:
output = _

D  = output[1]

R = {x for x in range(1,n) if D[variables[(x,0)]]}
Bl = {x for x in range(1,n) if D[variables[(x,1)]]}
Bd = {x for x in range(1,n) if D[variables[(x,2)]]}


def adder(A,B,p):
    # why did I use loops here? I am lazy...
    out = set()
    for a in A:
        for b in B:
            out.add((a+b)%p)
    return out
    
I = {0}
Div = set(range(1,n))

RplusR = adder(R,R,n)
print ('RplusR', RplusR == I|Div)
BlplusBl = adder(Bl,Bl,n)
print ('BlplusBl', BlplusBl == R|Bl)
BdplusBd = adder(Bd,Bd,n)
print ('BdplusBd', BdplusBd == R|Bd)
BlplusBd = adder(Bl,Bd,n)
print ('BlplusBd', BlplusBd == I|R|Bl|Bd)

TypeError: 'NoneType' object is not subscriptable

In [41]:
# n=29

print('R =', sorted(R))
print('B =', sorted(Bl))
print('Bconv =', sorted(Bd))

R = [1, 4, 5, 6, 7, 9, 13, 16, 20, 22, 23, 24, 25, 28]
B = [2, 3, 11, 14, 17, 19, 21]
Bconv = [8, 10, 12, 15, 18, 26, 27]


In [38]:
# n=38

print('R =', sorted(R))
print('B =', sorted(Bl))
print('Bconv =', sorted(Bd))

R = [1, 5, 6, 8, 9, 14, 16, 17, 18, 19, 20, 21, 22, 24, 29, 30, 32, 33, 37]
B = [3, 7, 10, 11, 13, 23, 26, 34, 36]
Bconv = [2, 4, 12, 15, 25, 27, 28, 31, 35]


In [44]:
# n=39

print('R =', sorted(R))
print('B =', sorted(Bl))
print('Bconv =', sorted(Bd))

R = [3, 5, 6, 7, 8, 12, 13, 16, 17, 19, 20, 22, 23, 26, 27, 31, 32, 33, 34, 36]
B = [1, 2, 4, 11, 15, 21, 25, 29, 30]
Bconv = [9, 10, 14, 18, 24, 28, 35, 37, 38]


In [48]:
# n=41

print('R =', sorted(R))
print('B =', sorted(Bl))
print('Bconv =', sorted(Bd))

R = [1, 3, 4, 5, 8, 11, 12, 13, 16, 18, 19, 22, 23, 25, 28, 29, 30, 33, 36, 37, 38, 40]
B = [2, 6, 10, 17, 20, 26, 27, 32, 34]
Bconv = [7, 9, 14, 15, 21, 24, 31, 35, 39]


In [52]:
# n=43
print(n)
print('R =', sorted(R))
print('B =', sorted(Bl))
print('Bconv =', sorted(Bd))

43
R = [1, 3, 4, 5, 8, 11, 12, 13, 14, 15, 18, 25, 28, 29, 30, 31, 32, 35, 38, 39, 40, 42]
B = [2, 6, 7, 9, 16, 22, 23, 24, 26, 33]
Bconv = [10, 17, 19, 20, 21, 27, 34, 36, 37, 41]


In [55]:
# n=44
print(n)
print('R =', sorted(R))
print('B =', sorted(Bl))
print('Bconv =', sorted(Bd))

44
R = [1, 2, 3, 4, 5, 6, 7, 13, 15, 16, 20, 22, 24, 28, 29, 31, 37, 38, 39, 40, 41, 42, 43]
B = [8, 12, 14, 17, 23, 25, 26, 33, 34, 35]
Bconv = [9, 10, 11, 18, 19, 21, 27, 30, 32, 36]


In [58]:
# n=45
print(n)
print('R =', sorted(R))
print('B =', sorted(Bl))
print('Bconv =', sorted(Bd))

45
R = [2, 4, 5, 7, 11, 12, 14, 15, 18, 19, 20, 22, 23, 25, 26, 27, 30, 31, 33, 34, 38, 40, 41, 43]
B = [1, 10, 13, 21, 28, 29, 36, 37, 39, 42]
Bconv = [3, 6, 8, 9, 16, 17, 24, 32, 35, 44]


In [61]:
# n=46
print(n)
print('R =', sorted(R))
print('B =', sorted(Bl))
print('Bconv =', sorted(Bd))

46
R = [1, 2, 6, 7, 8, 9, 10, 11, 14, 15, 20, 21, 23, 25, 26, 31, 32, 35, 36, 37, 38, 39, 40, 44, 45]
B = [12, 13, 16, 19, 24, 28, 29, 41, 42, 43]
Bconv = [3, 4, 5, 17, 18, 22, 27, 30, 33, 34]


In [63]:
# n=47
print(n)
print('R =', sorted(R))
print('B =', sorted(Bl))
print('Bconv =', sorted(Bd))

47
R = [1, 2, 6, 7, 8, 9, 10, 11, 14, 15, 20, 21, 23, 25, 26, 31, 32, 35, 36, 37, 38, 39, 40, 44, 45]
B = [12, 13, 16, 19, 24, 28, 29, 41, 42, 43]
Bconv = [3, 4, 5, 17, 18, 22, 27, 30, 33, 34]


In [66]:
# n=48
print(n)
print('R =', sorted(R))
print('B =', sorted(Bl))
print('Bconv =', sorted(Bd))

48
R = [1, 2, 6, 7, 9, 12, 15, 16, 17, 19, 20, 21, 22, 24, 26, 27, 28, 29, 31, 32, 33, 36, 39, 41, 42, 46, 47]
B = [3, 4, 8, 11, 13, 14, 18, 25, 38, 43]
Bconv = [5, 10, 23, 30, 34, 35, 37, 40, 44, 45]


In [69]:
# n=49
print(n)
print('R =', sorted(R))
print('B =', sorted(Bl))
print('Bconv =', sorted(Bd))

49
R = [2, 5, 6, 7, 9, 12, 13, 14, 15, 16, 18, 19, 20, 24, 25, 29, 30, 31, 33, 34, 35, 36, 37, 40, 42, 43, 44, 47]
B = [1, 17, 23, 27, 28, 38, 39, 41, 45, 46]
Bconv = [3, 4, 8, 10, 11, 21, 22, 26, 32, 48]


In [72]:
# n=50
print(n)
print('R =', sorted(R))
print('B =', sorted(Bl))
print('Bconv =', sorted(Bd))

50
R = [4, 6, 7, 8, 9, 10, 15, 16, 17, 20, 21, 22, 24, 25, 26, 28, 29, 30, 33, 34, 35, 40, 41, 42, 43, 44, 46]
B = [11, 12, 13, 18, 23, 31, 36, 45, 47, 48, 49]
Bconv = [1, 2, 3, 5, 14, 19, 27, 32, 37, 38, 39]


In [75]:
# n=51
print(n)
print('R =', sorted(R))
print('B =', sorted(Bl))
print('Bconv =', sorted(Bd))

51
R = [1, 6, 7, 8, 10, 12, 14, 16, 17, 18, 19, 20, 23, 24, 27, 28, 31, 32, 33, 34, 35, 37, 39, 41, 43, 44, 45, 50]
B = [2, 4, 5, 9, 11, 13, 15, 22, 26, 30, 48]
Bconv = [3, 21, 25, 29, 36, 38, 40, 42, 46, 47, 49]


In [78]:
# n=52
print(n)
print('R =', sorted(R))
print('B =', sorted(Bl))
print('Bconv =', sorted(Bd))

52
R = [1, 2, 6, 7, 8, 11, 13, 17, 18, 19, 20, 21, 24, 25, 26, 27, 28, 31, 32, 33, 34, 35, 39, 41, 44, 45, 46, 50, 51]
B = [3, 5, 10, 14, 15, 29, 30, 36, 40, 43, 48]
Bconv = [4, 9, 12, 16, 22, 23, 37, 38, 42, 47, 49]


In [81]:
# n=53
print(n)
print('R =', sorted(R))
print('B =', sorted(Bl))
print('Bconv =', sorted(Bd))

53
R = [1, 3, 4, 5, 7, 8, 11, 12, 15, 16, 17, 18, 19, 20, 23, 30, 33, 34, 35, 36, 37, 38, 41, 42, 45, 46, 48, 49, 50, 52]
B = [10, 13, 22, 24, 25, 26, 32, 39, 44, 47, 51]
Bconv = [2, 6, 9, 14, 21, 27, 28, 29, 31, 40, 43]


In [84]:
# n=54
print(n)
print('R =', sorted(R))
print('B =', sorted(Bl))
print('Bconv =', sorted(Bd))

54
R = [3, 4, 6, 8, 9, 10, 12, 15, 16, 18, 19, 21, 23, 24, 27, 30, 31, 33, 35, 36, 38, 39, 42, 44, 45, 46, 48, 50, 51]
B = [5, 11, 13, 20, 22, 25, 26, 37, 40, 47, 52, 53]
Bconv = [1, 2, 7, 14, 17, 28, 29, 32, 34, 41, 43, 49]


In [87]:
# n=55
print(n)
print('R =', sorted(R))
print('B =', sorted(Bl))
print('Bconv =', sorted(Bd))

55
R = [3, 4, 7, 9, 10, 12, 13, 15, 17, 19, 20, 22, 25, 26, 27, 28, 29, 30, 33, 35, 36, 38, 40, 42, 43, 45, 46, 48, 51, 52]
B = [5, 21, 23, 31, 37, 39, 41, 44, 47, 49, 53, 54]
Bconv = [1, 2, 6, 8, 11, 14, 16, 18, 24, 32, 34, 50]


In [90]:
# n=56
print(n)
print('R =', sorted(R))
print('B =', sorted(Bl))
print('Bconv =', sorted(Bd))

56
R = [2, 5, 6, 8, 10, 12, 14, 17, 18, 19, 20, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 36, 37, 38, 39, 42, 44, 46, 48, 50, 51, 54]
B = [3, 7, 9, 11, 15, 16, 21, 22, 43, 52, 55]
Bconv = [1, 4, 13, 34, 35, 40, 41, 45, 47, 49, 53]


In [93]:
# n=57
print(n)
print('R =', sorted(R))
print('B =', sorted(Bl))
print('Bconv =', sorted(Bd))

57
R = [3, 4, 5, 8, 11, 12, 14, 17, 18, 19, 21, 24, 25, 26, 27, 30, 31, 32, 33, 36, 38, 39, 40, 43, 45, 46, 49, 52, 53, 54]
B = [1, 2, 7, 9, 10, 16, 20, 22, 23, 29, 42, 44, 51]
Bconv = [6, 13, 15, 28, 34, 35, 37, 41, 47, 48, 50, 55, 56]


In [96]:
# n=58
print(n)
print('R =', sorted(R))
print('B =', sorted(Bl))
print('Bconv =', sorted(Bd))

58
R = [4, 5, 7, 8, 9, 12, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 36, 37, 38, 46, 49, 50, 51, 53, 54]
B = [1, 2, 3, 6, 18, 19, 35, 41, 42, 43, 44, 45, 47, 48]
Bconv = [10, 11, 13, 14, 15, 16, 17, 23, 39, 40, 52, 55, 56, 57]


In [99]:
# n=59
print(n)
print('R =', sorted(R))
print('B =', sorted(Bl))
print('Bconv =', sorted(Bd))

59
R = [1, 2, 3, 5, 7, 8, 12, 14, 15, 18, 19, 20, 22, 23, 26, 27, 32, 33, 36, 37, 39, 40, 41, 44, 45, 47, 51, 52, 54, 56, 57, 58]
B = [6, 9, 13, 17, 24, 28, 30, 34, 38, 43, 48, 49, 55]
Bconv = [4, 10, 11, 16, 21, 25, 29, 31, 35, 42, 46, 50, 53]


In [102]:
# n=60
print(n)
print('R =', sorted(R))
print('B =', sorted(Bl))
print('Bconv =', sorted(Bd))

60
R = [2, 3, 4, 6, 10, 12, 18, 20, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 36, 37, 38, 40, 42, 48, 50, 54, 56, 57, 58]
B = [1, 9, 11, 17, 25, 39, 41, 44, 45, 46, 47, 52, 53, 55]
Bconv = [5, 7, 8, 13, 14, 15, 16, 19, 21, 35, 43, 49, 51, 59]


In [105]:
# n=61
print(n)
print('R =', sorted(R))
print('B =', sorted(Bl))
print('Bconv =', sorted(Bd))

61
R = [2, 6, 8, 9, 10, 11, 12, 13, 16, 18, 20, 21, 22, 24, 27, 28, 29, 30, 31, 32, 33, 34, 37, 39, 40, 41, 43, 45, 48, 49, 50, 51, 52, 53, 55, 59]
B = [1, 5, 15, 19, 35, 36, 38, 44, 47, 54, 57, 58]
Bconv = [3, 4, 7, 14, 17, 23, 25, 26, 42, 46, 56, 60]


In [108]:
# n=62
print(n)
print('R =', sorted(R))
print('B =', sorted(Bl))
print('Bconv =', sorted(Bd))

62
R = [2, 3, 6, 8, 9, 12, 14, 15, 16, 17, 18, 21, 22, 23, 24, 26, 28, 29, 31, 33, 34, 36, 38, 39, 40, 41, 44, 45, 46, 47, 48, 50, 53, 54, 56, 59, 60]
B = [1, 4, 5, 10, 11, 13, 20, 25, 30, 35, 43, 55]
Bconv = [7, 19, 27, 32, 37, 42, 49, 51, 52, 57, 58, 61]
